In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
from time import time
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import pickle

from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('hcare.csv')

FileNotFoundError: ignored

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data = data[['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
       'Available Extra Rooms in Hospital', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Visitors with Patient', 'Age', 'Admission_Deposit', 'Stay']]

In [ ]:
data['Stay'].replace('More than 100 Days', '>100', inplace=True)

In [ ]:
data = data.drop_duplicates()


In [ ]:
data['Stay']= data['Stay'].replace({'0-10':0, '11-20':0, '21-30':1, '31-40':1, '41-50':1, '51-60':2,'61-70':2,'71-80':2,'81-90':2,'91-100':2,'>100':2})

In [ ]:
data.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
cat_cols = list(data.select_dtypes(include=['object']).columns)
for col in cat_cols:
    data[col] = label.fit_transform(data[col])

In [ ]:
data

In [ ]:
X[:,9:10]

In [ ]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 5,stratify=Y)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# X_train[['Visitors with Patient','Admission_Deposit']] = sc.fit_transform(X_train[['Visitors with Patient','Admission_Deposit']])
# X_test[['Visitors with Patient','Admission_Deposit']] = sc.transform(X_test[['Visitors with Patient','Admission_Deposit']])

X_train[:,9:10] = sc.fit_transform(X_train[:,9:10])
X_train[:,11:12] = sc.fit_transform(X_train[:,11:12])


X_train[:,9:10] = sc.transform(X_train[:,9:10])
X_train[:,11:12] = sc.transform(X_train[:,11:12])


In [ ]:
values= [RandomForestClassifier(), KNeighborsClassifier(), LogisticRegression(), DecisionTreeClassifier(), GaussianNB(), XGBClassifier()]

keys= ['RandomForestClassifier',
      'KNeighborsClassifier', 
       'LogisticRegression', 
       'DecisionTreeClassifier', 
       'GaussianNB',
      'XGBoost']

models= dict(zip(keys,values))
accuracy_scores=[]
train_times=[]

for key,value in models.items():
    t = time()
    value.fit(X_train,Y_train)
    duration = (time() - t) / 60
    Y_pred= value.predict(X_test)
    accuracy= accuracy_score(Y_test, Y_pred)
    accuracy_scores.append(accuracy)
    train_times.append(duration)   
    print(key)
    print(round(accuracy * 100, 2))

In [ ]:
values= [RandomForestClassifier(), KNeighborsClassifier(), LogisticRegression(), DecisionTreeClassifier(), GaussianNB(), XGBClassifier()]

keys= ['RandomForestClassifier',
      'KNeighborsClassifier', 
       'LogisticRegression', 
       'DecisionTreeClassifier', 
       'GaussianNB',
      'XGBoost']

models= dict(zip(keys,values))
accuracy_scores=[]
train_times=[]

for key,value in models.items():
    t = time()
    value.fit(X_train,Y_train)
    duration = (time() - t) / 60
    Y_pred= value.predict(X_train)
    accuracy= accuracy_score(Y_train, Y_pred)
    accuracy_scores.append(accuracy)
    train_times.append(duration)   
    print(key)
    print(round(accuracy * 100, 2))

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

param_space = {
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.1, prior='log-uniform'),
    'n_estimators': Integer(50, 200),
    'min_child_weight': Integer(1, 10),
    'gamma': Real(0, 1),
    'subsample': Real(0.5, 1),
    'colsample_bytree': Real(0.5, 1),
    'reg_alpha': Real(0, 1),
    'reg_lambda': Real(0, 1)
}

# Create an XGBoost classifier
xgb_model = XGBClassifier()

# Use BayesSearchCV to search for the best hyperparameters
bayes_search = BayesSearchCV(estimator=xgb_model, search_spaces=param_space, cv=5, n_jobs=-1)
bayes_search.fit(X_train, Y_train)

# Print the best hyperparameters and the corresponding mean cross-validation score
print("Best hyperparameters:", bayes_search.best_params_)
print("Best cross-validation score:", bayes_search.best_score_)

In [ ]:
model = XGBClassifier(learning_rate = 0.1, max_depth = 7, min_child_weight = 1, n_estimators = 200)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(accuracy_score(Y_train, Y_pred))

In [ ]:

X_train = X_train.astype(np.int64)

In [ ]:
X_train.dtypes

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold

# Load the dataset
iris = load_iris()
X = iris.data
y = iris.target

from skopt.space import Real, Integer

param_space = {
    'max_depth': Integer(3, 10, dtype=int),
    'learning_rate': Real(0.01, 0.1, prior='log-uniform', dtype=float),
    'n_estimators': Integer(50, 200, dtype=int),
    'min_child_weight': Integer(1, 10, dtype=int),
    'gamma': Real(0, 1, dtype=float),
    'subsample': Real(0.5, 1, dtype=float),
    'colsample_bytree': Real(0.5, 1, dtype=float),
    'reg_alpha': Real(0, 1, dtype=float),
    'reg_lambda': Real(0, 1, dtype=float)
}

# Define the XGBoost classifier with multiclass objective
xgb = XGBClassifier(objective='multi:softmax', num_class=3)

# Define the search strategy
search = BayesSearchCV(xgb, param_space, scoring='accuracy', n_jobs=-1, cv=StratifiedKFold(n_splits=5), n_iter=50)

# Run the search
search.fit(X, y)


In [ ]:
import xgboost as xgb
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3)

param_space = {
    'max_depth': Integer(3, 10, dtype=np.int64),
    'learning_rate': Real(0.01, 0.1, prior='log-uniform', dtype=float),
    'n_estimators': Integer(50, 200, dtype=int),
    'min_child_weight': Integer(1, 10, dtype=int),
    'gamma': Real(0, 1, dtype=float),
    'subsample': Real(0.5, 1, dtype=float),
    'colsample_bytree': Real(0.5, 1, dtype=float),
    'reg_alpha': Real(0, 1, dtype=float),
    'reg_lambda': Real(0, 1, dtype=float)
}

opt = BayesSearchCV(
    xgb_clf,
    param_space,
    cv=5,
    n_iter=10,
    n_jobs=-1
)

opt.fit(X, y)


In [ ]:
from bayes_opt import BayesianOptimization
import xgboost as xgb

def bayes_opt(X_train, y_train, param_space):
    dtrain = xgb.DMatrix(X_train, label=y_train)

    def xgb_evaluate(max_depth, n_estimators, min_child_weight):
        params = {'eval_metric': 'mlogloss',
                  'objective': 'multi:softprob',
                  'num_class': 4,
                  'max_depth': int(max_depth),
                  'n_estimators': int(n_estimators),
                  'min_child_weight': int(min_child_weight),
                  'silent': 1}

        cv_result = xgb.cv(params, dtrain, num_boost_round=1000, nfold=3,
                           seed=42, early_stopping_rounds=50)
        return -1.0 * cv_result['test-mlogloss-mean'].iloc[-1]

    optimizer = BayesianOptimization(xgb_evaluate, param_space)
    optimizer.maximize(init_points=10, n_iter=25)
    return optimizer


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

# Generate some synthetic data
X, y = make_classification(n_samples=1000, n_classes=3, n_features=10, n_informative=5, random_state=42)

# Define the search space for hyperparameters
param_space = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 200),
    'min_child_weight': (1, 10),
    'gamma': (0, 1),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1)
}

# Define the objective function to maximize
def xgb_cv(max_depth, learning_rate, n_estimators, min_child_weight, gamma, subsample, colsample_bytree, reg_alpha, reg_lambda):
    params = {'eval_metric': 'mlogloss',
              'max_depth': int(max_depth),
              'learning_rate': learning_rate,
              'n_estimators': int(n_estimators),
              'min_child_weight': int(min_child_weight),
              'gamma': gamma,
              'subsample': subsample,
              'colsample_bytree': colsample_bytree,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'objective': 'multi:softmax',
              'num_class': 3,
              'tree_method': 'gpu_hist'
             }
    
    # Cross-validation with XGBoost
    cv_result = xgb.cv(params, xgb.DMatrix(X, label=y), num_boost_round=1000, nfold=5)
    
    # Return the maximum mean validation accuracy
    return -1.0 * cv_result['test-mlogloss-mean'].iloc[-1]

# Perform Bayesian optimization
xgb_bo = BayesianOptimization(f=xgb_cv, pbounds=param_space, random_state=42)
xgb_bo.maximize(init_points=10, n_iter=20)

# Print the best hyperparameters
print(xgb_bo.max['params'])


In [ ]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, StratifiedKFold

# # Load iris dataset
# iris = load_iris()
# X, y = iris.data, iris.target

# Define the XGBoost model with parameters to optimize
def xgb_model(max_depth, learning_rate, n_estimators, gamma, subsample, colsample_bytree, reg_alpha, reg_lambda, min_child_weight):
    model = xgb.XGBClassifier(
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        min_child_weight=min_child_weight,
        objective='multi:softmax',
        random_state=42,
        num_class=len(set(Y_train))
    )
    return cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=StratifiedKFold()).mean()

# Define the hyperparameters search space
params = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 200),
    'gamma': (0, 1),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
    'reg_alpha': (0, 1),
    'reg_lambda': (0, 1),
    'min_child_weight': (0, 10)
}

# Create a BayesianOptimization object with the xgb_model function and the params search space
# optimizer = BayesianOptimization(
#     f=xgb_model,
#     pbounds=params,
#     random_state=42,
# )

bayes_search = BayesSearchCV(estimator=xgb_model, search_spaces=params, cv=5, n_jobs=-1)


# Run the optimization
# optimizer.maximize(n_iter=10)

# Print the best parameters found and the corresponding accuracy
# print(optimizer.max)


In [ ]:
model = XGBClassifier(colsample_bytree= 1, learning_rate = 0.1, max_depth = 10, min_child_weight = 1, n_estimators = 50,gamma = 0.9)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)
print(accuracy_score(Y_test, y_pred))

In [ ]:
Y_test

In [ ]:
set(y_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

model = XGBClassifier()

param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50,100, 500, 1000],
    'gamma': [0.01, 0.1, 0.5, 1],
    'subsample': [0.5, 1],
    'colsample_bytree': [0.5, 1, 1.5, 2]
}

grid_search = GridSearchCV(model, param_grid=param_grid,
                           cv=3, n_jobs=-1)

grid_search.fit(X_train, Y_train)